In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
pip install tensorflow-addons

In [ ]:
import datetime
import warnings
warnings.filterwarnings('ignore')
import torch
from glob import glob
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import numpy as np
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import KFold
import time
# from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
# from torch_poly_lr_decay import PolynomialLRDecay
import random
from torchvision import models
from torchsummary import summary as summary_
from sklearn.metrics import accuracy_score, log_loss, f1_score
import math
import librosa



os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'
torch.set_num_threads(8)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
VERSION = "20211005"
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0   113k      0 --:--:-- --:--:-- --:--:--  113k
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20211005 ...
Copying gs://tpu-pytorch/wheels/torch-nightly+20211005-cp37-cp37m-linux_x86_64.whl...
\ [1 files][145.7 MiB/145.7 MiB]                                                
Operation completed over 1 objects/145.7 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20211005-cp37-cp37m-linux_x86_64.whl...
\ [1 files][137.9 MiB/137.9 MiB]                                                
Operation completed over 1 objects/137.9 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-nightly+20211005-cp37-cp37m-linux_x86_64.whl...
/ [1 files][  5.7 MiB/  5.7 MiB]                                             

In [ ]:
train_normal_paths = sorted(glob("/content/gdrive/MyDrive/Dataset/leak_prediction/train/normal/*.wav"))
train_leak_paths = sorted(glob("/content/gdrive/MyDrive/Dataset/leak_prediction/train/leak/*.wav"))
test_paths = sorted(glob("/content/gdrive/MyDrive/Dataset/leak_prediction/test/*.wav"))
# england_train_paths = sorted(glob("dataset/train/england/*.wav"))
# hongkong_train_paths = sorted(glob("dataset/train/hongkong/*.wav"))
# us_train_paths = sorted(glob("dataset/train/us/*.wav"))
# test_paths = [f'dataset/test/{k+1}.wav' for k in range(6100)]

In [ ]:
len(train_normal_paths), len(train_leak_paths), len(test_paths)

(5170, 1596, 2958)

In [ ]:
def load_data(paths):
    result = []
    for path in tqdm(paths):
        # sr = 16000이 의미하는 것은 1초당 16000개의 데이터를 샘플링 한다는 것입니다.
        data, sr = librosa.load(path, sr = 16000)
        result.append(data)
    result = np.array(result, dtype = np.float32)
    # 메모리가 부족할 때는 데이터 타입을 변경해 주세요 ex) np.array(data, dtype = np.float32)

    return result

def get_feature(data, sr = 16000, n_fft = 2048, win_length = 200, hop_length = 160, n_mels = 64):
    mel = []
    for i in tqdm(data):
        # win_length 는 음성을 작은 조각으로 자를때 작은 조각의 크기입니다.
        # hop_length 는 음성을 작은 조각으로 자를때 자르는 간격을 의미합니다.
        # n_mels 는 적용할 mel filter의 개수입니다.
        mel_ = librosa.feature.melspectrogram(i, sr = sr, n_fft = n_fft, win_length = win_length, hop_length = hop_length, n_mels = n_mels)
        mel.append(mel_)
    mel = np.array(mel)
    mel = librosa.power_to_db(mel, ref = np.max)

#     mel_mean = mel.mean()
#     mel_std = mel.std()
#     mel = (mel - mel_mean) / mel_std

    return mel


def set_length(data, d_mini):

    result = []
    for value in tqdm(data):
        value = value[:d_mini]
        if len(value)<d_mini:
            value = np.append(value, [0]*(d_mini-len(value)))
        result.append(value)
    result = np.array(result)

    return result

In [ ]:
train_normal_data = load_data(train_normal_paths)
train_leak_data = load_data(train_leak_paths)
test_data = load_data(test_paths)

100%|██████████| 2958/2958 [29:53<00:00,  1.65it/s]


In [ ]:
np.save("/content/gdrive/MyDrive/Dataset/leak_prediction/train_normal_npy", train_normal_data)
np.save("/content/gdrive/MyDrive/Dataset/leak_prediction/train_leak_npy", train_leak_data)
np.save("/content/gdrive/MyDrive/Dataset/leak_prediction/test_npy", test_data)

In [ ]:
train_normal_data = np.load('/content/gdrive/MyDrive/Dataset/leak_prediction/train_normal_npy.npy', allow_pickle = True)
train_leak_data = np.load('/content/gdrive/MyDrive/Dataset/leak_prediction/train_leak_npy.npy', allow_pickle = True)
test_data = np.load('/content/gdrive/MyDrive/Dataset/leak_prediction/test_npy.npy', allow_pickle = True)

In [ ]:
train_data_list = [train_normal_data, train_leak_data]

In [ ]:
train_x = np.concatenate(train_data_list, axis=0)
train_x = set_length(train_x, 100000)

train_x_200 = get_feature(data = train_x, win_length=200)
train_x_400 = get_feature(data = train_x, win_length=400)
train_x_800 = get_feature(data = train_x, win_length=800)
train_x_1000 = get_feature(data = train_x, win_length=1000)


train_x_200 = train_x_200.reshape(train_x_200.shape[0], train_x_200.shape[1], train_x_200.shape[2], 1)
train_x_400 = train_x_400.reshape(train_x_400.shape[0], train_x_400.shape[1], train_x_400.shape[2], 1)
train_x_800 = train_x_800.reshape(train_x_800.shape[0], train_x_800.shape[1], train_x_800.shape[2], 1)
train_x_1000 = train_x_1000.reshape(train_x_1000.shape[0], train_x_1000.shape[1], train_x_1000.shape[2], 1)

train_x_multi = np.concatenate([train_x_200,
                                train_x_400,
                                train_x_800,
                                train_x_1000], -1)
np.save('/content/gdrive/MyDrive/Dataset/leak_prediction/train_x_multi.npy', train_x_multi)

100%|██████████| 6766/6766 [02:54<00:00, 38.82it/s]


In [ ]:
test_x = np.load('/content/gdrive/MyDrive/Dataset/leak_prediction/test_npy.npy', allow_pickle=True)
test_x = set_length(test_x, 100000)
# test_x_200 = get_feature(data = test_x, win_length=200)
test_x_400 = get_feature(data = test_x, win_length=400)
# test_x_800 = get_feature(data = test_x, win_length=800)
test_x_1000 = get_feature(data = test_x, win_length=1000)

# test_x_200 = test_x_200.reshape(test_x_200.shape[0], test_x_200.shape[1], test_x_200.shape[2], 1)
test_x_400 = test_x_400.reshape(test_x_400.shape[0], test_x_400.shape[1], test_x_400.shape[2], 1)
# test_x_800 = test_x_800.reshape(test_x_800.shape[0], test_x_800.shape[1], test_x_800.shape[2], 1)
test_x_1000 = test_x_1000.reshape(test_x_1000.shape[0], test_x_1000.shape[1], test_x_1000.shape[2], 1)
test_x_multi = np.concatenate([test_x_200,
                                test_x_400,
                                test_x_800,
                                test_x_1000], -1)
np.save('/content/gdrive/MyDrive/Dataset/leak_prediction/test_x_multi.npy', test_x_multi)


100%|██████████| 2958/2958 [01:09<00:00, 42.87it/s]


In [ ]:
train_x = np.load('/content/gdrive/MyDrive/Dataset/leak_prediction/train_x_multi.npy')

In [ ]:
test_x = np.load('/content/gdrive/MyDrive/Dataset/leak_prediction/test_x_multi.npy')

In [ ]:
len(train_normal_paths), len(train_leak_paths), len(test_paths)

(5170, 1596, 2958)

In [ ]:
train_y = np.concatenate((np.zeros(len(train_normal_paths), dtype = np.int),
                        np.ones(len(train_leak_paths), dtype = np.int)), axis = 0)

# train_y = np.concatenate((np.zeros(len(africa_train_data), dtype = np.int),
#                         np.ones(len(australia_train_data), dtype = np.int),
#                          np.ones(len(canada_train_data), dtype = np.int) * 2,
#                          np.ones(len(england_train_data), dtype = np.int) * 3,
#                          np.ones(len(hongkong_train_data), dtype = np.int) * 4,
#                          np.ones(len(us_train_data), dtype = np.int) * 5), axis = 0)

In [ ]:
train_y

array([0, 0, 0, ..., 1, 1, 1])

# Pytorch model building

In [ ]:
class conv_bn_relu(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(conv_bn_relu, self).__init__()
        
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=kernel_size // 2)
        self.BN = nn.BatchNorm2d(out_channels)
        self.Leakyrelu = nn.LeakyReLU(0.1)
        
        
    def forward(self, x):
        x = self.conv(x)
        x = self.BN(x)
        x = self.Leakyrelu(x)
        return x
    

class Network(nn.Module):
    def __init__(self, N):
        super(Network, self).__init__()
        self.N = N
        self.AveragePooling = nn.AvgPool2d(2)
        self.MaxPooling = nn.MaxPool2d(2)
        
        
        self.input_conv = conv_bn_relu(in_channels=4, out_channels=self.N, kernel_size=3)
        
        self.block1 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
            conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3),
        )
        
        self.block2 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
            conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3),
        )
        
        self.block3 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
            conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3),
        )
        
        self.block4 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
            conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3),
        )
        
        self.pool_block = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            nn.Dropout(0.15),
            nn.AvgPool2d(2),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*2, kernel_size=3),
            nn.Dropout(0.15),
            nn.AvgPool2d(2),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*2, kernel_size=3),
            nn.Dropout(0.15),
            nn.AdaptiveAvgPool2d(1)
        )
        
        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=self.N*2, out_features=2),
        )
        
    def forward(self, x, out=''):
        x = self.input_conv(x)
        x = self.MaxPooling(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.pool_block(x)
        x= self.output(x)
        
        if out=='sigmoid':
            x = F.sigmoid(x)
        return x

class U_network(nn.Module):
    def __init__(self, N):
        super(Network, self).__init__()
        self.N = N
        self.AveragePooling = nn.AvgPool2d(2)
        self.MaxPooling = nn.MaxPool2d(2)
        
        
        self.input_conv = conv_bn_relu(in_channels=4, out_channels=self.N, kernel_size=3)
        
        # self.up_block1 = nn.Sequential(
        #     conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
        #     conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
        #     conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
        # )
        
        # self.up_block2 = nn.Sequential(
        #     conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
        #     conv_bn_relu(in_channels=self.N*4, out_channels=self.N*8, kernel_size=3),
        #     conv_bn_relu(in_channels=self.N*8, out_channels=self.N*4, kernel_size=3),
        # )

        # self.up_block3 = nn.Sequential(
        #     conv_bn_relu(in_channels=self.N*4, out_channels=self.N*8, kernel_size=3),
        #     conv_bn_relu(in_channels=self.N*8, out_channels=self.N*16, kernel_size=3),
        # )
        
        # self.down_block3 = nn.Sequential(
        #     conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
        #     conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
        #     conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
        #     conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3),
        # )
        
        # self.down_block4 = nn.Sequential(
        #     conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
        #     conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
        #     conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
        #     conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3),
        # )
        self.up_block1 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
        )
        self.up_block2 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
        )
        self.up_block3 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*4, out_channels=self.N*8, kernel_size=3),
        )
        self.up_block4 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*8, out_channels=self.N*16, kernel_size=3),
        )
        self.up_block5 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*16, out_channels=self.N*32, kernel_size=3),
        )
        self.up_block1 = conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3)
        self.up_block2 = conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3)
        self.up_block3 = conv_bn_relu(in_channels=self.N*4, out_channels=self.N*8, kernel_size=3)
        self.up_block4 = conv_bn_relu(in_channels=self.N*8, out_channels=self.N*16, kernel_size=3)
        self.up_block5 = conv_bn_relu(in_channels=self.N*16, out_channels=self.N*32, kernel_size=3)
        self.up_block5 = conv_bn_relu(in_channels=self.N*32, out_channels=self.N*64, kernel_size=3)

        self.down_block1 = conv_bn_relu(in_channels=self.N*64, out_channels=self.N*32, kernel_size=3)
        self.down_block2 = conv_bn_relu(in_channels=self.N*32, out_channels=self.N*16, kernel_size=3)
        self.down_block3 = conv_bn_relu(in_channels=self.N*16, out_channels=self.N*8, kernel_size=3)
        self.down_block4 = conv_bn_relu(in_channels=self.N*8, out_channels=self.N*4, kernel_size=3)
        self.down_block5 = conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3)
        self.down_block6 = conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3)
  
        self.pool_block = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            nn.Dropout(0.15),
            nn.AvgPool2d(2),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*2, kernel_size=3),
            nn.Dropout(0.15),
            nn.AvgPool2d(2),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*2, kernel_size=3),
            nn.Dropout(0.15),
            nn.AdaptiveAvgPool2d(1)
        )
        
        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=self.N*2, out_features=2),
        )
        
    def forward(self, x, out=''):
        x = self.input_conv(x)
        x = self.MaxPooling(x)

        up1_out = self.up_block1(x)
        up2_out = self.up_block2(up1_out)
        up3_out = self.up_block3(up2_out)
        up4_out = self.up_block4(up3_out)
        up5_out = self.up_block5(up4_out)
        up5_out = self.up_block6(up5_out)

        down1_out = self.down_block1(up6_out)
        skipblock1 = down1_out + up5_out
        down2_out = self.down_block1(skipblock1)
        skipblock2 = down2_out + up4_out
        down3_out = self.down_block2(skipblock2)
        skipblock3 = down3_out + up3_out
        down4_out = self.down_block3(skipblock3)
        skipblock4 = down4_out + up2_out
        down5_out = self.down_block4(skipblock4)
        skipblock5 = down5_out + up1_out
        down6_out = self.down_block5(skipblock5)

        
        x = self.pool_block(x)
        x= self.output(x)
        
        if out=='sigmoid':
            x = F.sigmoid(x)
        return x
    

class VoiceDatasetSimple(Dataset):
        def __init__(self, X, y, transform, inference=False, roll=False):
            self.X = X
            self.y = y
            self.transform = transform
            self.inference = inference
            self.roll = roll
        def __len__(self):
            return len(self.X)
        
        def __getitem__(self, idx):
            X = self.X[idx]
            X = (X-train_x_min)/(train_x_max-train_x_min)
            
            if self.inference:
                X = self.transform(X)
                return X
            else:
                if (self.roll==True) and (random.randint(0, 1)==1):
                    X = np.roll(X,random.randint(-200, 200), axis=1)
                    
                X = self.transform(X)    
                y = self.y[idx]
                
                onehot = np.zeros(6)
                onehot[y] = 1.
                y = onehot
                return X, y

def model_save(model, path):
    torch.save({
        'model': model.state_dict(),
    }, path)
    

In [ ]:
model = Network(4)
summary_(model, (4,4,4))

In [ ]:
!nvidia-smi

Thu Nov  4 11:49:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    40W / 300W |   3705MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm

ModuleNotFoundError: ignored

In [ ]:
train_x = np.load('/content/gdrive/MyDrive/Dataset/leak_prediction/train_x_multi.npy')
train_y = np.concatenate((np.zeros(len(train_normal_paths), dtype = np.int),
                          np.ones(len(train_leak_paths), dtype = np.int)), axis = 0)
train_x_min = train_x.min()
train_x_max = train_x.max()

In [ ]:
# idx = [k+person for k in range(0, len(train_x), 3)][:-1]
# train_x=train_x[idx]
# train_y=train_y[idx]


skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
folds=[]
for train_idx, valid_idx in skf.split(train_x, train_y):
    folds.append((train_idx, valid_idx))

        
for fold in range(5):
    torch.cuda.empty_cache() 
    epochs=35
    batch_size=8
    model_name = f'network-epoch{epochs}-fold({fold}).pth'
    train_idx, valid_idx = folds[fold]

    transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    train_dataset = VoiceDatasetSimple(X=train_x[train_idx], y=train_y[train_idx], transform=transform, roll=False)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

    valid_dataset = VoiceDatasetSimple(X=train_x[valid_idx], y=train_y[valid_idx], transform=transform)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False, num_workers=8)


    # model compile
    model = Network(32).to(device)
    # model = nn.DataParallel(model, device_ids=[0,1,2])
    
    
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr =1e-3)
    Q = math.floor(len(train_dataset)/batch_size+1)*epochs/7
    lrs = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = Q)
    
    # loss
    criterion = nn.CrossEntropyLoss()


    best = 9999
    for epoch in range(epochs):
        start = time.time()
        model.train()
        train_loss=0
        train_pred_list=[]
        train_true_list=[]
        train_log_loss=0
        for X, y in (train_loader):
            X = torch.tensor(X, dtype=torch.float32, device=device)
            y = torch.tensor(y, dtype=torch.float32 , device=device)

            optimizer.zero_grad()
            pred = model(X)
            loss = criterion(pred, y.argmax(1))
            loss.backward()
            optimizer.step()
            lrs.step()

            train_pred_list += F.softmax(pred).argmax(1).detach().cpu().numpy().tolist()
            train_true_list += y.argmax(1).detach().cpu().numpy().tolist()
            train_loss+=loss.item()
        train_accuracy=accuracy_score(train_true_list, train_pred_list)


        
        with torch.no_grad():
            model.eval()
            valid_loss=0
            valid_log_loss=0
            valid_pred_list=[]
            valid_true_list=[]
            for X, y in (valid_loader):
                X = torch.tensor(X, dtype=torch.float32, device=device)
                y = torch.tensor(y, dtype=torch.float32 , device=device)

                pred = model(X)
                loss = criterion(pred, y.argmax(1))

                valid_pred_list += F.softmax(pred).argmax(1).detach().cpu().numpy().tolist()
                valid_true_list += y.argmax(1).detach().cpu().numpy().tolist()
                valid_loss+=loss.item()

        valid_accuracy=accuracy_score(valid_true_list, valid_pred_list)

        if valid_loss/len(valid_loader) < best:
            model_save(model, f'/content/gdrive/MyDrive/Dataset/leak_prediction/model_day2/unet_{model_name}')
            best = valid_loss/len(valid_loader)

        print(f'===================== Epoch : {epoch+1}/{epochs}    time : {time.time()-start:.0f}s =====================')
        print(f'TRAIN -> loss : {train_loss/len(train_loader):.5f}     accuracy : {train_accuracy:.5f}')
        print(f'VALID -> loss : {valid_loss/len(valid_loader):.5f}     accuracy : {valid_accuracy:.5f}    best : {best:.5f}\n\n')

===================== Epoch : 1/35    time : 63s =====================
TRAIN -> loss : 0.42795     accuracy : 0.81264
VALID -> loss : 0.81437     accuracy : 0.80281    best : 0.81437


===================== Epoch : 2/35    time : 63s =====================
TRAIN -> loss : 0.40610     accuracy : 0.82095
VALID -> loss : 0.35930     accuracy : 0.85377    best : 0.35930


===================== Epoch : 3/35    time : 63s =====================
TRAIN -> loss : 0.36177     accuracy : 0.83999
VALID -> loss : 0.86347     accuracy : 0.50222    best : 0.35930


===================== Epoch : 4/35    time : 62s =====================
TRAIN -> loss : 0.33391     accuracy : 0.85772
VALID -> loss : 0.59384     accuracy : 0.78360    best : 0.35930


===================== Epoch : 5/35    time : 63s =====================
TRAIN -> loss : 0.32918     accuracy : 0.86567
VALID -> loss : 0.63019     accuracy : 0.78360    best : 0.35930


===================== Epoch : 6/35    time : 63s =====================
TRAI

In [ ]:
X_test = np.load('/content/gdrive/MyDrive/Dataset/leak_prediction/test_x_multi.npy')

test_dataset = VoiceDatasetSimple(X=X_test, y=None, transform=transform, inference=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=30, shuffle=False, num_workers=8)

In [ ]:
result=0
for person in range(3):
    for fold in range(5):
        with torch.no_grad():
            weights = torch.load(f'/content/gdrive/MyDrive/Dataset/leak_prediction/model_day2/N=32_network-epoch{epochs}-person({person})-fold({fold}).pth')
            model.load_state_dict(weights['model'])
            model.eval()
            preds = []
            for X in tqdm(test_loader):
                X = torch.tensor(X, dtype=torch.float32, device=device)
                pred = F.softmax(model(X)).detach().cpu().numpy().tolist()
                preds+=pred
        preds = np.array(preds)
        result+=preds

100%|██████████| 99/99 [00:08<00:00, 11.77it/s]


In [ ]:
result.shape

(2958, 2)

In [ ]:
result = result[:,:2]

In [ ]:
result[:,:2]

array([[0.00270645, 0.00039331],
       [0.00269099, 0.00042851],
       [0.00275095, 0.00033468],
       ...,
       [0.00220097, 0.00195711],
       [0.00169846, 0.00350212],
       [0.00134062, 0.00471375]])

In [ ]:
result[:,0], result[:,1] = result[:,0] / len(train_normal_paths), result[:,1] / len(train_leak_paths)

In [ ]:
result[:,0], result[:,1] = result[:,0] * len(train_normal_paths), result[:,1] * len(train_leak_paths)

In [ ]:
k = result.argmax(1)

In [ ]:
len(k[k == 1]), len(k[k == 0]), len(k)

(1439, 1519, 2958)

In [ ]:
len(k[k == 1]), len(k[k == 0]), len(k)

(1112, 1846, 2958)

In [ ]:
csv = pd.read_csv('/content/gdrive/MyDrive/Dataset/leak_prediction/output_sample.csv')

In [ ]:
csv.iloc[:,1] = result.argmax(1)

In [ ]:
csv

,AudioName,ClassID
0,audio_00000.wav,0
1,audio_00001.wav,0
2,audio_00002.wav,0
3,audio_00003.wav,0
4,audio_00004.wav,1
...,...,...
2953,audio_02953.wav,0
2954,audio_02954.wav,0
2955,audio_02955.wav,0
2956,audio_02956.wav,1


In [ ]:
csv

,AudioName,ClassID
0,audio_00000.wav,0
1,audio_00001.wav,0
2,audio_00002.wav,0
3,audio_00003.wav,0
4,audio_00004.wav,1
...,...,...
2953,audio_02953.wav,0
2954,audio_02954.wav,0
2955,audio_02955.wav,1
2956,audio_02956.wav,1


In [ ]:
csv

,AudioName,ClassID
0,audio_00000.wav,0
1,audio_00001.wav,0
2,audio_00002.wav,0
3,audio_00003.wav,0
4,audio_00004.wav,1
...,...,...
2953,audio_02953.wav,0
2954,audio_02954.wav,0
2955,audio_02955.wav,0
2956,audio_02956.wav,1


In [ ]:
csv.to_csv('/content/test_N=32base.csv', index = False)

# Tensorflow model building

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Convolution2D, BatchNormalization, Flatten,
                                     Dropout, Dense, AveragePooling2D, Add)
from tensorflow.keras.callbacks import EarlyStopping
# import tensorflow_addons as tfa


In [ ]:
def block(input_, units = 32, dropout_rate = 0.5):
    
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(input_)
    x = BatchNormalization()(x)
    x_res = x
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Add()([x, x_res])
    x = AveragePooling2D()(x)
    x = Dropout(rate=dropout_rate)(x)
    
    return x

def second_block(input_, units = 64, dropout_rate = 0.5):
    
    x = Convolution2D(units, 1, padding ="same", activation = "relu")(input_)
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = Convolution2D(units * 4, 1, padding ="same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x_res = x
    x = Convolution2D(units, 1, padding ="same", activation = "relu")(x)
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = Convolution2D(units * 4, 1, padding ="same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Convolution2D(units, 1, padding = "same", activation = "relu")(x)
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = Convolution2D(units * 4, 1, padding = "same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Add()([x, x_res])
    x = AveragePooling2D()(x)
    x = Dropout(rate=dropout_rate)(x)
    
    return x

In [ ]:
def build_fn():
    dropout_rate = 0.3
    
    in_ = Input(shape = (train_x.shape[1:]))
    
    block_01 = block(in_, units = 32, dropout_rate = dropout_rate)
    block_02 = block(block_01, units = 64, dropout_rate = dropout_rate)
    block_03 = block(block_02, units = 128, dropout_rate = dropout_rate)

    block_04 = second_block(block_03, units = 64, dropout_rate = dropout_rate)
    block_05 = second_block(block_04, units = 128, dropout_rate = dropout_rate)

    x = Flatten()(block_05)

    x = Dense(units = 64, activation = "relu")(x)
    x = BatchNormalization()(x)
    x_res = x
    x = Dropout(rate = dropout_rate)(x)

    x = Dense(units = 64, activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Add()([x_res, x])
    x = Dropout(rate = dropout_rate)(x)

    model_out = Dense(units = 1, activation = 'sigmoid')(x)
    model = Model(in_, model_out)
    return model

In [ ]:
model = build_fn()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 626, 4)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 626, 32)  1184        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 626, 32)  128         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 626, 32)  9248        batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
train_x = np.load('/content/gdrive/MyDrive/Dataset/leak_prediction/train_x_multi.npy')
train_y = np.concatenate((np.zeros(len(train_normal_paths), dtype = np.int),
                          np.ones(len(train_leak_paths), dtype = np.int)), axis = 0)
train_x_min = train_x.min()
train_x_max = train_x.max()

In [ ]:
split = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 10)

pred = []
pred_ = []
count = 0
for train_idx, val_idx in split.split(train_x, train_y):
    count += 1
    x_train, y_train = train_x[train_idx], train_y[train_idx]
    x_val, y_val = train_x[val_idx], train_y[val_idx]
    model = build_fn()
    model.compile(optimizer = keras.optimizers.Adam(0.002),
                  loss = 'binary_crossentropy', # keras.losses.SparseCategoricalCrossentropy()
                #  loss = tf.keras.metrics.Accuracy(),tfa.metrics.F1Score(num_classes = 2)
                 metrics = ['acc'])
    callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath = f'/content/gdrive/MyDrive/Dataset/leak_prediction/model/cnn_base_{count}.h5',
        monitor = 'val_loss',
        save_best_only=True
    )]

    model.fit(x = x_train, y = y_train, validation_data = (x_val, y_val), epochs = 8, callbacks = callbacks_list)
    # print("*******************************************************************")
    # pred.append(model.predict(test_x))
    # pred_.append(np.argmax(model.predict(test_x), axis = 1))
    print("*******************************************************************")

Epoch 1/8
170/170 [==============================] - 36s 96ms/step - loss: 0.5179 - acc: 0.7971 - val_loss: 0.7588 - val_acc: 0.7762
Epoch 2/8
170/170 [==============================] - 14s 83ms/step - loss: 0.3903 - acc: 0.8424 - val_loss: 0.5644 - val_acc: 0.7917
Epoch 3/8
170/170 [==============================] - 14s 83ms/step - loss: 0.3556 - acc: 0.8466 - val_loss: 0.3679 - val_acc: 0.8397
Epoch 4/8
170/170 [==============================] - 14s 83ms/step - loss: 0.3219 - acc: 0.8683 - val_loss: 0.3321 - val_acc: 0.8678
Epoch 5/8
170/170 [==============================] - 14s 83ms/step - loss: 0.3148 - acc: 0.8697 - val_loss: 1.0565 - val_acc: 0.7637
Epoch 6/8
170/170 [==============================] - 14s 83ms/step - loss: 0.3093 - acc: 0.8731 - val_loss: 1.1632 - val_acc: 0.7637
Epoch 7/8
170/170 [==============================] - 14s 83ms/step - loss: 0.3066 - acc: 0.8742 - val_loss: 0.7588 - val_acc: 0.7770
Epoch 8/8
170/170 [==============================] - 14s 84ms/step - 

InternalError: ignored

In [ ]:
globals()

{'Add': keras.layers.merge.Add,
 'AveragePooling2D': keras.layers.pooling.AveragePooling2D,
 'BatchNormalization': keras.layers.normalization.batch_normalization.BatchNormalization,
 'Convolution2D': keras.layers.convolutional.Conv2D,
 'DataLoader': torch.utils.data.dataloader.DataLoader,
 'Dataset': torch.utils.data.dataset.Dataset,
 'Dense': keras.layers.core.Dense,
 'Dropout': keras.layers.core.Dropout,
 'EarlyStopping': keras.callbacks.EarlyStopping,
 'F': <module 'torch.nn.functional' from '/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py'>,
 'Flatten': keras.layers.core.Flatten,
 'In': ['',
  "get_ipython().magic('pip install tensorflow-addons')",
  'import datetime\nimport warnings\nwarnings.filterwarnings(\'ignore\')\nimport torch\nfrom glob import glob\nimport os\nfrom torch.utils.data import Dataset, DataLoader\nimport pandas as pd\nfrom tqdm import tqdm\nimport numpy as np\nfrom torchvision import transforms\nimport torchvision.models as models\nimport torch.nn 

In [ ]:
clear(x_train), clear(y_train), clear(x_val),clear(y_val)

In [ ]:
    model = build_fn()
    model.compile(optimizer = keras.optimizers.Adam(0.002), # keras.losses.SparseCategoricalCrossentropy()
                 loss = tf.keras.metrics.Accuracy(),
                 metrics = [tfa.metrics.F1Score(num_classes = 2)])

    history = model.fit(x = x_train, y = y_train, validation_data = (x_val, y_val), epochs = 8)